In [3]:
class Node:
    
    def __init__(self, name, parents, cpt):
        self.name = name
        self.parents = parents
        self.cpt = cpt

    def __str__(self):
        return self.name

In [13]:
class CustomBayesModel:

    def __init__(self, nodes):
        self.nodes = nodes

    def get_node(self, name):
        for node in self.nodes:
            if node.name == name:
                return node
        return None

    def get_parents(self, node):
        parents = []
        for parent_name in node.parents:
            parent = self.get_node(parent_name)
            if parent:
                parents.append(parent)
        return parents

    def get_children(self, node):
        children = []
        for child in self.nodes:
            if node.name in child.parents:
                children.append(child)
        return children

    def add_edge(self, parent, child):
        child.parents.append(parent.name)

    def remove_edge(self, parent, child):
        child.parents.remove(parent.name)

    def get_ancestors(self, node):
        ancestors = []
        parents = self.get_parents(node)
        for parent in parents:
            ancestors.append(parent)
            ancestors.extend(self.get_ancestors(parent))
        return ancestors

    def get_descendants(self, node):
        descendants = []
        children = self.get_children(node)
        for child in children:
            descendants.append(child)
            descendants.extend(self.get_descendants(child))
        return descendants

    def get_independent_nodes(self, node):
        independent_nodes = []
        for n in self.nodes:
            if n not in self.get_ancestors(node) and n not in self.get_descendants(node) and n != node:
                independent_nodes.append(n)
        return independent_nodes

    def get_conditional_probability(self, node, evidence):
        parents = self.get_parents(node)
        if len(parents) == 0:
            return node.cpt[0][1]
        else:
            for row in node.cpt:
                if all(row[i] == evidence[parent.name] for i, parent in enumerate(parents)):
                    return row[-1]
        return None

    def get_probability(self, node, evidence):
        parents = self.get_parents(node)
        if len(parents) == 0:
            return node.cpt[0][1]
        else:
            for row in node.cpt:
                if all(row[i] == evidence[parent.name] for i, parent in enumerate(parents)):
                    return row[-1]
        return None

    def get_probability_of_evidence(self, evidence):
        probability = 1
        for node in self.nodes:
            if node.name in evidence:
                probability *= self.get_conditional_probability(node, evidence)
        return probability

    def get_probability_of_evidence_given(self, evidence, node):
        probability = 1
        for n in self.nodes:
            if n.name in evidence:
                probability *= self.get_probability(n, evidence)
        return probability

    def bayes(self, p_a, p_b, p_b_a):
        return (p_b_a * p_a) / p_b

    def calculate_inference_markov_chain(self, target_node, evidence):
        probability = 1

        # Iterate over the ancestors of the target node
        for ancestor in self.get_ancestors(target_node):
            # Calculate the conditional probability based on the evidence and ancestor's parents
            parents = self.get_parents(ancestor)
            parents_evidence = {parent.name: evidence.get(parent.name, None) for parent in parents}
            conditional_prob = self.get_conditional_probability(ancestor, parents_evidence)

            # Multiply the conditional probability to the running total
            if conditional_prob is not None:
                probability *= conditional_prob

        # Calculate the conditional probability of the target node based on its parents and evidence
        parents = self.get_parents(target_node)
        parents_evidence = {parent.name: evidence.get(parent.name, None) for parent in parents}
        conditional_prob_target = self.get_conditional_probability(target_node, parents_evidence)

        # Multiply the conditional probability of the target node to the running total
        if conditional_prob_target is not None:
            probability *= conditional_prob_target

        return probability

    def variable_elimination(self, variables, evidence):
        # Sort variables based on the number of parents (ascending order)
        variables = sorted(variables, key=lambda x: len(x.parents))

        # Initialize a factor for each variable
        factors = {variable.name: variable.cpt for variable in variables}

        # Iterate through the variables and eliminate them
        for variable in variables:
            # Check if the variable is not in evidence
            if variable.name not in evidence:
                # Sum out the variable from the factors
                new_factor = []
                for value in variable.values:
                    sum_prob = sum(factors[variable.name][value] for variable in variables if factors[variable.name][value] is not None)
                    new_factor.append(sum_prob)
                factors[variable.name] = new_factor

        # Multiply the remaining factors to calculate the final probability
        probability = 1
        for factor in factors.values():
            if factor is not None:
                probability *= factor

        return probability

    def inference(self, evidence):
        play_tennis_node = self.get_node('PlayTennis')

        # Variables to eliminate (excluding PlayTennis and evidence variables)
        variables_to_eliminate = [
            node for node in self.nodes if node.name != 'PlayTennis' and node.name not in evidence]

        # Calculate the probability of PlayTennis given the evidence using variable elimination
        probability_play_tennis = self.variable_elimination(
            variables_to_eliminate, evidence)

        # Calculate the probability of PlayTennis using the Markov chain rule
        probability_play_tennis_markov = self.calculate_inference_markov_chain(
            play_tennis_node, evidence)

        # Combine the probabilities using Bayes' rule
        final_probability = self.bayes(
            probability_play_tennis, 1, probability_play_tennis_markov)

        return 'Yes' if final_probability > 0.5 else 'No'

In [14]:
newBayesModel = CustomBayesModel([
    Node('Outlook', [], [
        [['Sunny'], 5/14],
        [['Overcast'], 4/14],
        [['Rain'], 5/14]
    ]),
    Node('Temperature', [], [
        [['Hot'], 4/14],
        [['Mild'], 6/14],
        [['Cool'], 4/14]
    ]),
    Node('Humidity', [], [
        [['High'], 7/14],
        [['Normal'], 7/14]
    ]),
    Node('Wind', [], [
        [['Weak'], 8/14],
        [['Strong'], 6/14]
    ]),
    Node('PlayTennis', ['Outlook', 'Temperature', 'Humidity', 'Wind'], [
        [['Sunny', 'Hot', 'High', 'Weak', 'Yes'], 2/9],
        [['Sunny', 'Hot', 'High', 'Strong', 'No'], 0/9],
        [['Overcast', 'Hot', 'High', 'Weak', 'Yes'], 4/9],
        [['Rain', 'Mild', 'High', 'Weak', 'Yes'], 3/9],
        [['Rain', 'Cool', 'Normal', 'Weak', 'Yes'], 2/9],
        [['Rain', 'Cool', 'Normal', 'Strong', 'No'], 0/9],
        [['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'], 4/9],
        [['Sunny', 'Mild', 'High', 'Weak', 'No'], 0/9],
        [['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'], 3/9],
        [['Rain', 'Mild', 'Normal', 'Weak', 'Yes'], 2/9],
        [['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'], 6/9],
        [['Overcast', 'Mild', 'High', 'Strong', 'Yes'], 4/9],
        [['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'], 4/9],
        [['Rain', 'Mild', 'High', 'Strong', 'No'], 0/9]
    ])
])

In [17]:
print(newBayesModel.inference({
    'Outlook': 'Sunny',
    'Temperature': 'Cool',
    'Humidity': 'Normal',
    'Wind': 'Strong'
}), '\n')


No 

